# Synthetic experiment

This notebook has code for the synthetic experiment of Section 3.4.

The implementation of the risk matrix score here uses Probability of Exceedance (PoE) forecasts to determine the severity category. The PoE forecasts of forecasters who don't follow the warning directive are adjusted so that they lie in the certainty category that they actually forecast.


In [1]:
import xarray as xr
import numpy as np
from scipy.stats import norm
import scores

# local module
import my_functions as mf

In [2]:
# functions for remapping probabilities

def remap(values, old_lower, old_upper, new_lower, new_upper):
    """
    Linear map mapping old_lower -> new_lower, old_upper -> new_upper.
    Values outside [old_lower, old_upper] are mapped to NaN.
    """
    values = values.where(values >= old_lower).where(values <= old_upper)
    gradient = (new_upper - new_lower) / (old_upper - old_lower)
    intercept = new_lower - gradient * old_lower
    result = gradient * values + intercept
    return result

def multi_remap(values, old_cut_points, new_cut_points):
    """
    Repeated application of `remap`.
    Args:
        values: data array of values to remap
        old_cut_points: list of points
        new_cut_points: list of points to which old_cut_points are remapped
    Returns:
        data array of new values
    """
    n = len(old_cut_points)

    if len(new_cut_points) != n:
        raise Exception('old_cut_points and new_cut_points must be the same length')

    result = xr.full_like(values, np.nan)
    for i in range(n-1):
        tmp =  remap(values, old_cut_points[i], old_cut_points[i+1], new_cut_points[i], new_cut_points[i+1])
        result = result.combine_first(tmp)
    return result

def playful_remap(poe, probability_thresholds, severity_thresholds):
    """
    Remaps probability to midpoint of alternate certainty category if warning level doesn't change.
    This function assumes the warning scaling of the risk matrix for the synthetic experiment in the paper.

    Args:
        poe: xarray array of probability of exceedence forecasts, with dimension `sev_threshold`
        probability_thresholds: probability thresholds for the warning service
        severity_thresholds: severity thresholds for the warning service.

    Returns:
        rejigged probability of exceedance forecasts matching PlayfulPranay's certainty category selection.
    """
    # isolate severity poe forecasts
    poe_mod = poe.sel(sev_threshold=slice(severity_thresholds[0], severity_thresholds[0]))
    poe_sev = poe.sel(sev_threshold=slice(severity_thresholds[1], severity_thresholds[1]))
    poe_ext = poe.sel(sev_threshold=slice(severity_thresholds[2], severity_thresholds[2]))

    # adjust MOD+ forecast
    mod0 = poe_mod < probability_thresholds[0]
    mod1 = (poe_mod < probability_thresholds[1]) & (poe_mod >= probability_thresholds[0])
    mod2 = (poe_mod < probability_thresholds[2]) & (poe_mod >= probability_thresholds[1])
    mod3 = poe_mod >= probability_thresholds[2]

    new_mod0 = xr.full_like(poe_mod, (probability_thresholds[0] + probability_thresholds[1])/2).where(mod0)
    new_mod1 = xr.full_like(poe_mod, (0 + probability_thresholds[0])/2).where(mod1)
    new_mod2 = xr.full_like(poe_mod, (1 + probability_thresholds[2])/2).where(mod2)
    new_mod3 = xr.full_like(poe_mod, (probability_thresholds[1] + probability_thresholds[2])/2).where(mod3)
    new_mod = new_mod0.combine_first(new_mod1).combine_first(new_mod2).combine_first(new_mod3)

    # adjust SEV+ forecast
    sev01 = poe_sev < probability_thresholds[1]
    sev2 = (poe_sev < probability_thresholds[2]) & (poe_sev >= probability_thresholds[1])
    sev3 = poe_sev >= probability_thresholds[2]

    new_sev01 = poe_sev.where(sev01)
    new_sev2 = xr.full_like(poe_sev, (1 + probability_thresholds[2])/2).where(sev2)
    new_sev3 = xr.full_like(poe_sev, (probability_thresholds[1] + probability_thresholds[2])/2).where(sev3)
    new_sev = new_sev01.combine_first(new_sev2).combine_first(new_sev3)

    # adjust EXT forecast
    ext0 = poe_ext < probability_thresholds[0]
    ext1 = (poe_ext < probability_thresholds[1]) & (poe_ext >= probability_thresholds[0])
    ext2 = (poe_ext < probability_thresholds[2]) & (poe_ext >= probability_thresholds[1])
    ext3 = poe_ext >= probability_thresholds[2]

    new_ext0 = poe_ext.where(ext0)
    new_ext1 = xr.full_like(poe_ext, (probability_thresholds[1] + probability_thresholds[2])/2).where(ext1)
    new_ext2 = xr.full_like(poe_ext, (probability_thresholds[0] + probability_thresholds[1])/2).where(ext2)
    new_ext3 = poe_ext.where(ext3)
    new_ext = new_ext0.combine_first(new_ext1).combine_first(new_ext2).combine_first(new_ext3)

    new_poe = xr.concat([new_mod, new_sev, new_ext], 'sev_threshold')
    return new_poe

In [3]:
# synthetic experiment set up

# probability thresholds for the warning service
PROB_THRESHOLDS = [.1, .3, .5]

# probability thresholds used by RiskAverseRick
RISK_AVERSE_PROB_THRESHOLDS = [.05, .2, .4]

# probability thresholds used by RiskTolerantRena
RISK_TOLERANT_PROB_THRESHOLDS = [.2, .4, .6]

# Severity thresholds for the warning service
SEV_THRESHOLDS = [35., 37., 40.]

# number of independent experiments (forecast cases)
N_FCST_CASE = 1000000

# mean (mu) and standard deviation (sigma) of seasonal, synoptic and mesoscale information
MU_SEASON = 20
SIGMA_SEASON = 10
MU_SYNOP = 0
SIGMA_SYNOP = 5
MU_MESO = 0
SIGMA_MESO = 2

# weights for each decision threshold for the warning scaling
WEIGHTS = np.array(
    [
        [0, 0, 1],  # high prob threshold
        [1, 1, 0],  # mod prob threshold
        [0, 1, 1],  # low prob threshold
    ]
)

# unform weights
UNIFORM = np.array(
    [
        [1, 1, 1],  # high prob threshold
        [1, 1, 1],  # mod prob threshold
        [1, 1, 1],  # low prob threshold
    ]
)

In [4]:
# generate y1, y2 and y3
y1 = xr.DataArray(
    data=norm.rvs(loc=MU_SEASON, scale=SIGMA_SEASON, size=N_FCST_CASE),
    dims=['fcst_case'],
    coords={'fcst_case': np.arange(N_FCST_CASE)}
)
y2 = xr.DataArray(
    data=norm.rvs(loc=MU_SYNOP, scale=SIGMA_SYNOP, size=N_FCST_CASE),
    dims=['fcst_case'],
    coords={'fcst_case': np.arange(N_FCST_CASE)}
)
y3 = xr.DataArray(
    data=norm.rvs(loc=MU_MESO, scale=SIGMA_MESO, size=N_FCST_CASE),
    dims=['fcst_case'],
    coords={'fcst_case': np.arange(N_FCST_CASE)}
)

# generate broadcast arrays for the random variables
da_sev_thresholds = xr.DataArray(
    data=SEV_THRESHOLDS,
    dims=["sev_threshold"],
    coords={"sev_threshold": SEV_THRESHOLDS},
)
y1, y2, y3, da_sev_thresholds = xr.broadcast(y1, y2, y3, da_sev_thresholds)

# calculate the observations in degrees Celsius
obs = y1 + y2 + y3

# we need to convert this to categorical observations: 1 if obs is in the severity category and 0 if not
obs = (obs > da_sev_thresholds).astype(int)

obs

<xarray.DataArray (fcst_case: 1000000, sev_threshold: 3)> Size: 24MB
array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [1, 0, 0],
       [0, 0, 0]])
Coordinates:
  * fcst_case      (fcst_case) int64 8MB 0 1 2 3 ... 999996 999997 999998 999999
  * sev_threshold  (sev_threshold) float64 24B 35.0 37.0 40.0

In [5]:
# calculate probability of exceedance forecasts for NeverWarnNate, SeasonalSam and SynopticSally

sally = xr.full_like(y1, np.nan)
sally.values = 1 - norm.cdf(da_sev_thresholds, loc=y1 + y2, scale=SIGMA_MESO)

sam = xr.full_like(y1, np.nan)
sam.values = 1 - norm.cdf(da_sev_thresholds, loc=y1, scale=np.sqrt(SIGMA_SYNOP ** 2 + SIGMA_MESO ** 2))

nate = xr.full_like(y1, MU_SEASON)
nate.values = 1 - norm.cdf(da_sev_thresholds, loc=nate, scale=np.sqrt(SIGMA_SEASON ** 2 + SIGMA_SYNOP ** 2 + SIGMA_MESO ** 2))

# we'll combine the forecasts into one data set
fcsts = xr.Dataset({
    "SynopticSally": sally,
    "SeasonalSam": sam,
    "NeverWarnNate": nate,
})

fcsts

<xarray.Dataset> Size: 80MB
Dimensions:        (fcst_case: 1000000, sev_threshold: 3)
Coordinates:
  * fcst_case      (fcst_case) int64 8MB 0 1 2 3 ... 999996 999997 999998 999999
  * sev_threshold  (sev_threshold) float64 24B 35.0 37.0 40.0
Data variables:
    SynopticSally  (fcst_case, sev_threshold) float64 24MB 0.0 0.0 ... 0.0 0.0
    SeasonalSam    (fcst_case, sev_threshold) float64 24MB 2.253e-12 ... 5.2e-09
    NeverWarnNate  (fcst_case, sev_threshold) float64 24MB 0.0933 ... 0.03913

In [6]:
# Adjust the probability of exceedance (PoE) forecasts of RiskAverseRick, RiskTolerantReena and PlayfulPranay
# so that the new PoEs fall in the certainty categories that they forecast.

fcsts['RiskAverseRick'] = multi_remap(sally, [0] + RISK_AVERSE_PROB_THRESHOLDS + [1], [0] + PROB_THRESHOLDS + [1])

# RiskAverseRick
fcsts['RiskTolerantReena'] = multi_remap(sally, [0] + RISK_TOLERANT_PROB_THRESHOLDS + [1], [0] + PROB_THRESHOLDS + [1])

# PlayfulPranay
fcsts['PlayfulPranay'] = playful_remap(sally, PROB_THRESHOLDS, SEV_THRESHOLDS)

fcsts

<xarray.Dataset> Size: 152MB
Dimensions:            (fcst_case: 1000000, sev_threshold: 3)
Coordinates:
  * fcst_case          (fcst_case) int64 8MB 0 1 2 3 ... 999997 999998 999999
  * sev_threshold      (sev_threshold) float64 24B 35.0 37.0 40.0
Data variables:
    SynopticSally      (fcst_case, sev_threshold) float64 24MB 0.0 0.0 ... 0.0
    SeasonalSam        (fcst_case, sev_threshold) float64 24MB 2.253e-12 ... ...
    NeverWarnNate      (fcst_case, sev_threshold) float64 24MB 0.0933 ... 0.0...
    RiskAverseRick     (fcst_case, sev_threshold) float64 24MB 0.0 0.0 ... 0.0
    RiskTolerantReena  (fcst_case, sev_threshold) float64 24MB 0.0 0.0 ... 0.0
    PlayfulPranay      (fcst_case, sev_threshold) float64 24MB 0.2 0.0 ... 0.0

In [7]:
# score the forecasters
warning_weights = mf.wt_matrix_to_xr(WEIGHTS, PROB_THRESHOLDS, SEV_THRESHOLDS)
uniform_weights = mf.wt_matrix_to_xr(UNIFORM, PROB_THRESHOLDS, SEV_THRESHOLDS)

risk_matrix_scores = mf.risk_matrix_score(fcsts, obs, uniform_weights, PROB_THRESHOLDS, 'sev_threshold')
warning_scores = mf.risk_matrix_score(fcsts, obs, warning_weights, PROB_THRESHOLDS, 'sev_threshold')

risk_matrix_scores

<xarray.Dataset> Size: 56MB
Dimensions:            (fcst_case: 1000000)
Coordinates:
  * fcst_case          (fcst_case) int64 8MB 0 1 2 3 ... 999997 999998 999999
Data variables:
    SynopticSally      (fcst_case) float64 8MB 0.0 0.0 0.0 0.0 ... 0.0 0.1 0.0
    SeasonalSam        (fcst_case) float64 8MB 0.0 0.0 0.0 0.0 ... 0.0 0.7 0.0
    NeverWarnNate      (fcst_case) float64 8MB 0.0 0.0 0.0 0.0 ... 0.0 2.1 0.0
    RiskAverseRick     (fcst_case) float64 8MB 0.0 0.0 0.0 0.0 ... 0.0 0.4 0.0
    RiskTolerantReena  (fcst_case) float64 8MB 0.0 0.0 0.0 0.0 ... 0.0 0.1 0.0
    PlayfulPranay      (fcst_case) float64 8MB 0.1 0.1 0.1 0.1 ... 0.1 0.6 0.1

In [8]:
# mean risk matrix score
risk_matrix_scores.mean('fcst_case')

<xarray.Dataset> Size: 48B
Dimensions:            ()
Data variables:
    SynopticSally      float64 8B 0.06558
    SeasonalSam        float64 8B 0.1889
    NeverWarnNate      float64 8B 0.4209
    RiskAverseRick     float64 8B 0.06856
    RiskTolerantReena  float64 8B 0.06897
    PlayfulPranay      float64 8B 0.218

In [9]:
# mean warning score
warning_scores.mean('fcst_case')

<xarray.Dataset> Size: 48B
Dimensions:            ()
Data variables:
    SynopticSally      float64 8B 0.03317
    SeasonalSam        float64 8B 0.09796
    NeverWarnNate      float64 8B 0.2273
    RiskAverseRick     float64 8B 0.03491
    RiskTolerantReena  float64 8B 0.03517
    PlayfulPranay      float64 8B 0.03317

In [9]:
# test for statistical significance: Sally vs Rick

diff_series = risk_matrix_scores['RiskAverseRick'] - risk_matrix_scores['SynopticSally']
diff_series = diff_series.assign_coords(h=1).expand_dims('h')
scores.stats.statistical_tests.diebold_mariano(diff_series, 'h', 'h', confidence_level=0.95)

# if ci_upper and ci_lower are both positive, then RiskAverseRick has a mean risk matrix score than
# SynopticSally and this is statistically sgnificant at the 5% level

<xarray.Dataset> Size: 56B
Dimensions:          (h: 1)
Coordinates:
  * h                (h) int64 8B 1
Data variables:
    mean             (h) float64 8B 0.002981
    dm_test_stat     (h) float64 8B 27.18
    timeseries_len   (h) int64 8B 1000000
    confidence_gt_0  (h) float64 8B 1.0
    ci_upper         (h) float64 8B 0.003196
    ci_lower         (h) float64 8B 0.002766